# RDS server user data cleaning
This notebook was created to take advantage of the Jupyter visualizations when cleaning the data.<br>
The commands defined in this notebook will be then used in the **clean_user_data** method of the **DataCleaning** class.

 Performs the cleaning of the user data for 

1. NULL values
2. errors with dates
3. incorrectly typed values
4. rows filled with the wrong information

In [1]:
# need to mount Google Drive to be able to load data from it
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [41]:
# import pandas and the RDS table that needs cleaning
import pandas as pd
df = pd.read_csv("/content/drive/MyDrive/001. Data Science/005. Ai Core/014. Multinational Data Centralization Project/RDS_table.csv")

In [42]:
df.head()
#df.columns

,Unnamed: 0,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
0,0,0,Sigfried,Noack,1990-09-30,Heydrich Junitz KG,rudi79@winkler.de,Zimmerstr. 1/0\n59015 Gießen,Germany,DE,+49(0) 047905356,2018-10-10,93caf182-e4e9-4c6e-bebb-60a1a9dcf9b8
1,1,1,Guy,Allen,1940-12-01,Fox Ltd,rhodesclifford@henderson.com,Studio 22a\nLynne terrace\nMcCarthymouth\nTF0 9GH,United Kingdom,GB,(0161) 496 0674,2001-12-20,8fe96c3a-d62d-4eb5-b313-cf12d9126a49
2,2,2,Harry,Lawrence,1995-08-02,"Johnson, Jones and Harris",glen98@bryant-marshall.co.uk,92 Ann drive\nJoanborough\nSK0 6LR,United Kingdom,GB,+44(0)121 4960340,2016-12-16,fc461df4-b919-48b2-909e-55c95a03fe6b
3,3,3,Darren,Hussain,1972-09-23,Wheeler LLC,daniellebryan@thompson.org,19 Robinson meadow\nNew Tracy\nW22 2QG,United Kingdom,GB,(0306) 999 0871,2004-02-23,6104719f-ef14-4b09-bf04-fb0c4620acb0
4,4,4,Garry,Stone,1952-12-20,Warner Inc,billy14@long-warren.com,3 White pass\nHunterborough\nNN96 4UE,United Kingdom,GB,0121 496 0225,2006-09-01,9523a6d3-b2dd-4670-a51a-36aebc89f579


In [43]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15320 entries, 0 to 15319
Data columns (total 13 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   Unnamed: 0     15320 non-null  int64 
 1   index          15320 non-null  int64 
 2   first_name     15299 non-null  object
 3   last_name      15299 non-null  object
 4   date_of_birth  15299 non-null  object
 5   company        15299 non-null  object
 6   email_address  15299 non-null  object
 7   address        15299 non-null  object
 8   country        15299 non-null  object
 9   country_code   15299 non-null  object
 10  phone_number   15299 non-null  object
 11  join_date      15299 non-null  object
 12  user_uuid      15299 non-null  object
dtypes: int64(2), object(11)
memory usage: 1.5+ MB


In [44]:
df.duplicated().sum()

0

In [45]:
df.isna().sum()

Unnamed: 0        0
index             0
first_name       21
last_name        21
date_of_birth    21
company          21
email_address    21
address          21
country          21
country_code     21
phone_number     21
join_date        21
user_uuid        21
dtype: int64

In [46]:
#pd.set_option('display.max_rows', 500)
#for name in df['first_name'].unique():
#    print(name)

In [47]:
# plot the "Unnamed" column
df["Unnamed: 0"].sample(100)

13176    13176
8344      8344
5550      5550
7930      7930
5820      5820
         ...  
9609      9609
6573      6573
740        740
15074    15074
494        494
Name: Unnamed: 0, Length: 100, dtype: int64

In [48]:
# plot the "level_0" column
df["index"].sample(100)

7352      7360
4260      4263
1988      1989
1100      1101
7647      7656
         ...  
14872    14899
13106    13130
13802    13826
2919      2920
11070    11084
Name: index, Length: 100, dtype: int64

## To does
1. drop the "Unnamed: 0" and "index" columns
2. drop the rows containing NaN
3. Convert the DOB in datetime and leverage on it to determine if any of the dates is wrong

In [49]:
df = df.drop(['Unnamed: 0', 'index'], axis = 1)

In [50]:
df = df[df.notna().any(axis=1)]

In [51]:
# not sure it is really worth modifying that
import datetime
df['date_of_birth'] = pd.to_datetime(df['date_of_birth'], infer_datetime_format=True, errors = 'coerce')

Remove the incorrect dates

In [52]:
df = df[df['date_of_birth'].notna()]

In [54]:
df['date_of_birth'].isna().sum()

0

DOB now contains zero errors

In [35]:
pd.to_datetime(df['join_date'], infer_datetime_format=True, errors = 'coerce').isna().sum()

15

In [36]:
df = df[df['join_date'].notna()]

In [39]:
df['date_of_birth'].isna().sum()

0

In [40]:
df['join_date'].isna().sum()

0

In [ ]:
df['country'].unique()

In [ ]:
df['country_code'].unique()

In [ ]:
# check if country and country_code coincide
print(len(df[(df['country'] == "Germany") & (df['country_code'] == "DE")]))
print(len(df[df['country'] == "Germany"]))
print(len(df[df['country_code'] == "DE"]))

In [ ]:
# check if country and country_code coincide
print(len(df[(df['country'] == "United Kingdom") & (df['country_code'] == "GB")]))
print(len(df[df['country'] == "United Kingdom"]))
print(len(df[df['country_code'] == "GB"]))

In [ ]:
# assign the correct country code
df.loc[(df['country'] == "United Kingdom") & (df['country_code'] != "GB"), 'country_code'] = 'GB'
# check if now the numbers are aligned
print(len(df[(df['country'] == "United Kingdom") & (df['country_code'] == "GB")]))
print(len(df[df['country'] == "United Kingdom"]))
print(len(df[df['country_code'] == "GB"]))

In [ ]:
print(len(df[(df['country'] == "United States") & (df['country_code'] == "US")]))
print(len(df[df['country'] == "United States"]))
print(len(df[df['country_code'] == "US"]))

In [ ]:
df['country'].value_counts()

In [ ]:
df[df['country'].isin(['I7G4DMDZOZ',
       'AJ1ENKS3QL', 'XGI7FM0VBJ', 'S0E37H52ON', 'XN9NGL5C0B',
       '50KUU3PQUF', 'EWE3U0DZIV', 'GMRBOMI0O1', 'YOTSVPRBQ7',
       '5EFAFD0JLI', 'PNRMPSYR1J', 'RQRB7RMTAD', '3518UD5CE8',
       '7ZNO5EBALT', 'T4WBZSW0XI'])]

In [ ]:
df[~df['country'].isin(['United Kingdom', 'Germany', 'United States'])]

These values are all wrong, let's drop them

In [ ]:
df = df[df['country'].isin(['United Kingdom', 'Germany', 'United States'])]

In [ ]:
df['country'].value_counts()

In [ ]:
df['country_code'].value_counts()

OK, I stop here! Tonight I better save the file!

In [ ]:
! pip install tabula-py

In [ ]:
import tabula
pdf_link = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf'
dfs = tabula.read_pdf(pdf_link, pages='all')

In [ ]:
# tabula extracts a list
type(dfs)

In [ ]:
len(dfs[0])

In [ ]:
import pandas as pd
df_2 = pd.concat(dfs)

In [ ]:
len(df_Final)

In [ ]:
# this is what I have to input into the function:
import tabula
pdf_link = 'https://data-handling-public.s3.eu-west-1.amazonaws.com/card_details.pdf'
dfs = tabula.read_pdf(pdf_link, pages='all')
df = pd.concat(dfs)

https://stackoverflow.com/questions/65626278/using-tabula-py-to-read-table-without-header-from-pdf-format

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
pd.set_option('display.max_rows', 500)
df.head(10)

In [ ]:
print((df['card_number']=='NULL').sum())
print((df['expiry_date']=='NULL').sum())
print((df['card_provider']=='NULL').sum())
print((df['date_payment_confirmed']=='NULL').sum())

it seems all the tables have the same number of nulls, 11 in total. Let's check they all belong to the same rows.

In [ ]:
df = df[df['card_number']!='NULL']

In [ ]:
df.duplicated().sum()

there are no duplicated rows

In [ ]:
df['card_provider'].unique()

array(['Diners Club / Carte Blanche', 'American Express', 'JCB 16 digit',
       'JCB 15 digit', 'Maestro', 'Mastercard', 'Discover',
       'VISA 19 digit', 'VISA 16 digit', 'VISA 13 digit'], dtype=object)

In [ ]:
df[df['card_provider'].isin(['NB71VBAHJE',
       'WJVMUO4QX6', 'JRPRLPIBZ2', 'TS8A81WFXV', 'JCQMU8FN85',
       '5CJH7ABGDR', 'DE488ORDXY', 'OGJTXI6X1H', '1M38DYQTZV',
       'DLWF2HANZF', 'XGZBYBYGUW', 'UA07L7EILH', 'BU9U947ZGV',
       '5MFWFBZRM9'])]

All the data with those weird card numbers are corrupted or unuseable. Let's remove them.

In [ ]:
df = df[df['card_provider'].isin(['Diners Club / Carte Blanche', 'American Express', 'JCB 16 digit',
       'JCB 15 digit', 'Maestro', 'Mastercard', 'Discover',
       'VISA 19 digit', 'VISA 16 digit', 'VISA 13 digit'])]

## So to summarize

In [ ]:
df = df[df['card_number']!='NULL']
df = df[df['card_provider'].isin(['Diners Club / Carte Blanche', 'American Express', 'JCB 16 digit',
       'JCB 15 digit', 'Maestro', 'Mastercard', 'Discover',
       'VISA 19 digit', 'VISA 16 digit', 'VISA 13 digit'])]